In [51]:
#https://www.kaggle.com/code/ashydv/sales-prediction-simple-linear-regression
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
# Import the numpy and pandas package
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
#import pandas_datareader.data as web
import seaborn as sns
import pandas as pd
import os.path
import numpy as np
from scipy.stats import spearmanr
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_squared_error,
                             mean_absolute_error,
                             mean_squared_log_error, 
                             median_absolute_error, 
                             explained_variance_score, 
                             r2_score)

In [52]:
start_time = time.time()

data = pd.DataFrame(pd.read_csv("housing.csv"))
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200


In [53]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [54]:
print(data.isnull().sum())

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
ocean_proximity         0
median_house_value      0
dtype: int64


In [55]:
data['total_bedrooms'].fillna(data['total_bedrooms'].median(), inplace=True)
print(data.isnull().sum())

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
ocean_proximity       0
median_house_value    0
dtype: int64


In [56]:
label_encoder = LabelEncoder()
data['ocean_proximity'] = label_encoder.fit_transform(data['ocean_proximity'])

names = ['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']
scaler = StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(data)
scaled_df = pd.DataFrame(data, columns=names)
print(scaled_df.head())

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                  41          880           129.0   
1    -122.22     37.86                  21         7099          1106.0   
2    -122.24     37.85                  52         1467           190.0   
3    -122.25     37.85                  52         1274           235.0   
4    -122.25     37.85                  52         1627           280.0   

   population  households  median_income  median_house_value  
0         322         126         8.3252              452600  
1        2401        1138         8.3014              358500  
2         496         177         7.2574              352100  
3         558         219         5.6431              341300  
4         565         259         3.8462              342200  


In [57]:
X = scaled_df.drop("median_house_value", axis=1)
# Our descriptive feature
print("X.describe()")
print(X.describe())

y = scaled_df['median_house_value']
print(y.describe())

X.describe()
          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.000000   
25%     -121.800000     33.930000           18.000000   1447.750000   
50%     -118.490000     34.260000           29.000000   2127.000000   
75%     -118.010000     37.710000           37.000000   3148.000000   
max     -114.310000     41.950000           52.000000  39320.000000   

       total_bedrooms    population    households  median_income  
count    20640.000000  20640.000000  20640.000000   20640.000000  
mean       536.838857   1425.476744    499.539680       3.870671  
std        419.391878   1132.462122    382.329753       1.899822  
min          1.000000      3.000000      1.000000       0.499900  
25%        2

In [58]:
print("\nLet's build our Linear regression Model\n")

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

print("\n(X_train.head())\n")
print("======================")
print(X_train.head())

regression = linear_model.LinearRegression()
regression.fit(X_train, Y_train)


Let's build our Linear regression Model


(X_train.head())

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
15961    -122.43     37.71                  52         1410           286.0   
1771     -122.35     37.95                  42         1485           290.0   
16414    -121.24     37.90                  16           50            10.0   
5056     -118.35     34.02                  34         5218          1576.0   
8589     -118.39     33.89                  38         1851           332.0   

       population  households  median_income  
15961         879         282         3.1908  
1771          971         303         3.6094  
16414          20           6         2.6250  
5056         3538        1371         1.5143  
8589          750         314         7.3356  


LinearRegression()

In [59]:
print("Model Evaluation")
print("R-squared: %.3f" % regression.score(X_test, Y_test))
Y_pred = regression.predict(X_test)
print("Mean squared error: %.3f" % mean_squared_error(Y_test, Y_pred))  # Test MSE
print("Mean absolute error: %.3f" % mean_absolute_error(Y_test, Y_pred))

end_time = time.time()
time_taken = end_time - start_time
time_taken_ms = (time_taken) * 1000
print(f"Time taken for the pipeline execution: {time_taken_ms:.2f} milliseconds, {time_taken:.4f} seconds")

Model Evaluation
R-squared: 0.628
Mean squared error: 4880637095.920
Mean absolute error: 50913.069
Time taken for the pipeline execution: 313.75 milliseconds, 0.3138 seconds


In [60]:
#regression.predict(np.array([[230,37.5,70.5]]))